In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
# from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
# from sklearn.tree import export_graphviz
# from sklearn.externals.six import StringIO
# import pydotplus
# from sklearn import tree
# import graphviz
# from sklearn.metrics import confusion_matrix
# from sklearn.utils.multiclass import unique_labels
# from sklearn.metrics import classification_report
# import pstats
# import cProfile
# from IPython.display import Image
# from sklearn.metrics import roc_curve, roc_auc_score
# from sklearn.metrics import precision_score, recall_score, roc_auc_score, roc_curve

In [2]:
df=pd.read_csv('analysis_rfm_1010nooutlier.csv')

In [3]:
df.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4534726 entries, 0 to 4534725
Data columns (total 42 columns):
Unnamed: 0                  4534726 non-null int64
userid                      4534726 non-null object
restart_number              4534726 non-null int64
map_name                    4534726 non-null object
difficulty                  4534726 non-null object
win_loss_quit               4534726 non-null object
round_reached               4534726 non-null int64
lives_left                  4534726 non-null int64
player_rank                 4534726 non-null int64
current_shards              4534726 non-null int64
current_gold                4534726 non-null int64
current_gems                4534726 non-null int64
recency_in_days             4534726 non-null int64
recency_in_weeks            4534726 non-null int64
active_status               4534726 non-null object
countofdays                 4534726 non-null int64
frequency_in_total          4534726 non-null int64
app_opened_perd

In [7]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
df['platform'] = le.fit_transform(df['platform'])
df['win_loss_quit'] = le.fit_transform(df['win_loss_quit'])
df['difficulty'] = le.fit_transform(df['difficulty'])
df['map_name'] = le.fit_transform(df['map_name'])
df['active_status'] = le.fit_transform(df['active_status'])
#df.head()

In [ ]:
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn.ensemble import RandomForestClassifier
feature_cols = ['avg_duration_mins','initial_2_weeks','current_gold','active_weeks','current_gems','current_shards','frequency_in_total','app_opened_perday_median','map_name','easy_count','initial_nappopen','n_games','initial_ngames','avg_duration_mins','difficulty_skull','easy_count','medium_count','hard_count','impoppable_count','active_weeks','initial_2_weeks','initial_nappopen','initial_ngames','wlqgtr_cnt','n_games','wlqgtr_cnt','lives_left' ]
X = df[feature_cols]# Features
y = df.active_status
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0) # 70% training and 30% test


In [10]:
from imblearn.over_sampling import SMOTE
feature_cols = ['avg_duration_mins','initial_2_weeks','current_gold','active_weeks','current_gems','current_shards','frequency_in_total','app_opened_perday_median','map_name','easy_count','initial_nappopen','n_games','initial_ngames','avg_duration_mins','difficulty_skull','easy_count','medium_count','hard_count','impoppable_count','active_weeks','initial_2_weeks','initial_nappopen','initial_ngames','wlqgtr_cnt','n_games','wlqgtr_cnt','lives_left' ]
X = df[feature_cols]# Features
y = df.active_status
X_resampled, y_resampled = SMOTE().fit_resample(X, y)
from collections import Counter
print(sorted(Counter(y_resampled).items()))
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.3, random_state=0)

[(0, 4284976), (1, 4284976), (2, 4284976)]


In [34]:
from sklearn.preprocessing import StandardScaler
scaled_X_train = preprocessing.StandardScaler().fit(X_train)
scaled_X_test = preprocessing.StandardScaler().fit(X_test)
#StandardScaler(copy=True, with_mean=True, with_std=True)

In [21]:
y_train[y_train==0].shape

(2998073,)

In [22]:
import numpy as np
idx=np.random.choice(y_train[y_train==0].shape[0],int(np.round(y_train.shape[0]*0.2)))
print(sorted(Counter(y_train[idx]).items()))

[(0, 600299), (1, 599873), (2, 599518)]


In [24]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.neighbors import KNeighborsClassifier
# Use the random grid to search for best hyperparameters
# First create the base model to tune
knn = KNeighborsClassifier()
parameters = {'n_neighbors': [3,5,11],'weights':['uniform','distance'],'metric':['euclidean','manhattan']}
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
knn_model = RandomizedSearchCV(estimator = knn, param_distributions = parameters, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
k=knn_model.fit(X_train[idx], y_train[idx])
print(k.best_score_)
print(k.best_estimator_)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  30 | elapsed: 52.0min remaining: 52.0min
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed: 71.0min finished


0.8059838083225445
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='manhattan',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='distance')


AttributeError: 'RandomizedSearchCV' object has no attribute 'best_param_'

In [26]:
print(k.best_estimator_)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='manhattan',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='distance')


In [27]:
Knn_1=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='manhattan',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='distance')

In [28]:
%timeit Knn_1.fit(X_train[idx], y_train[idx])

14min 53s ± 6.67 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
